Copyright (c) 2020. Cognitive Scale Inc. All rights reserved.
Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/7998b8a481fccd467463deb1fc46d19622079b0e/LICENSE.md)


# Building a scan programmatically with your own model

**In this notebook, we will be demonstrating how to create a scan in Certifai using your own model. We will show some examples of how to use models and datasets to run scans**

**Documentation for Certifai can be found at https://cognitivescale.github.io/cortex-certifai/docs/about**

**To begin, we will import the libraries required to run Certifai scans via Jupyter Lab**

In [1]:
import pandas as pd
import matplotlib as plt
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from copy import copy
import yaml

from certifai.scanner.builder import (CertifaiScanBuilder, CertifaiPredictorWrapper, CertifaiModel, CertifaiModelMetric,
                                      CertifaiDataset, CertifaiGroupingFeature, CertifaiDatasetSource,
                                      CertifaiPredictionTask, CertifaiTaskOutcomes, CertifaiOutcomeValue)
from certifai.scanner.report_utils import scores, construct_scores_dataframe

**For multiprocessing to work in a Notebook, we need the encoder to be outside of the notebook. This code imports the encoder from the common library in a way that works in hosted notebooks as well as locally.**

In [2]:
from certifai.common.utils.encoding import CatEncoder

# STEP (1): Setting up the dataset and model to be scanned

**Task 1): Setting up the dataset**

**Load the data into a DataFrame for use in both training and later analysing the model. In this example we use the German Credit dataset**

In [3]:
all_data_file = "datasets/german_credit_eval.csv"
df = pd.read_csv(all_data_file)
df.head()

,checkingstatus,duration,history,purpose,amount,savings,employ,installment,status,others,...,property,age,otherplans,housing,cards,job,liable,telephone,foreign,outcome
0,... >= 200 DM / salary assignments for at leas...,6,critical account/ other credits existing (not ...,car (new),1343,... < 100 DM,.. >= 7 years,1,male : single,others - none,...,real estate,> 25 years,none,own,2,skilled employee / official,2,phone - none,foreign - no,1
1,... < 0 DM,28,existing credits paid back duly till now,car (new),4006,... < 100 DM,1 <= ... < 4 years,3,male : single,others - none,...,"car or other, not in attribute 6",> 25 years,none,own,1,unskilled - resident,1,phone - none,foreign - yes,2
2,no checking account,24,existing credits paid back duly till now,radio/television,2284,... < 100 DM,4 <= ... < 7 years,4,male : single,others - none,...,"car or other, not in attribute 6",> 25 years,none,own,1,skilled employee / official,1,"phone - yes, registered under the customers name",foreign - yes,1
3,no checking account,24,existing credits paid back duly till now,radio/television,1533,... < 100 DM,... < 1 year,4,female : divorced/separated/married,others - none,...,"car or other, not in attribute 6",> 25 years,stores,own,1,skilled employee / official,1,"phone - yes, registered under the customers name",foreign - yes,1
4,no checking account,12,existing credits paid back duly till now,car (new),1101,... < 100 DM,1 <= ... < 4 years,3,male : married/widowed,others - none,...,real estate,> 25 years,none,own,2,skilled employee / official,1,"phone - yes, registered under the customers name",foreign - yes,1


**Task 2): Set the categorical columns of the dataset up for the encoder (in our case we will encapsulate this in the CatEncoder class, which may be found in the same directory as this notebook). We also note the column that contains the ground truth labels for training in 'label_column' (in this dataset this is 'outcome').**

In [4]:
cat_columns = [
    'checkingstatus',
    'history',
    'purpose',
    'savings',
    'employ',
    'status',
    'others',
    'property',
    'age',
    'otherplans',
    'housing',
    'job',
    'telephone',
    'foreign'
    ]

label_column = 'outcome'

**In our example we use a simple logistic classifier from sklearn. This is where you can add your own model. Rather than using the one provided, you can import and set up your model to be used here.**

**Note that the predictor must be picklable.  Specifically if the predictor class itself is defined in a notebook rather than an imported module then it *must* be in a different notebook file for Python muti-processing to handle it correctly**

**Task 3) Because the outcome column won't be presented to the model at prediction time we need to drop it from the dataset. We then split into a test and train set.**

In [5]:
y = df[label_column]
X = df.drop(label_column, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


**Task 4) Set the encoder**

In [6]:
encoder = CatEncoder(cat_columns, X)

**Task 5) Fit the classification model**

In [7]:
def build_model(data, name, test=None):
    if test is None:
        test = data
        

    parameters = {'C': (0.5, 1.0, 2.0), 'solver': ['lbfgs'], 'max_iter': [1000]}
    m = LogisticRegression()
    model = GridSearchCV(m, parameters, cv=3)
    model.fit(data[0], data[1])

    # Assess on the test data
    accuracy = model.score(test[0], test[1].values)
    print(f"Model '{name}' accuracy is {accuracy}")
    return model

logistic_model = build_model((encoder(X_train.values), y_train),
                        'Logistic classifier',
                        test=(encoder(X_test.values), y_test))

Model 'Logistic classifier' accuracy is 0.77


**Task 6) Wrap up the model and the encoder so that Certifai sees it as part of the model**

In [8]:
logistic_model_proxy = CertifaiPredictorWrapper(logistic_model, encoder=encoder)

**Task 7) Compute model's accuracy with the test dataset**

In [9]:
logistic_accuracy = logistic_model.score(encoder(X_test.values), y_test.values)
print(f"Logistic classifier model accuracy on test data is {logistic_accuracy}")

Logistic classifier model accuracy on test data is 0.77


# Step (2) Create the scan object using the ScanBuilder class

**To allow easy working with Certifai from notebooks, or other programmatic use cases, the `ScanBuilder` class abstracts the scan definition and provides an object model to manipulate it.  Building up a definition in this way allows either direct running of the scan in the notebook, or export as a scan definition file, which can be run by the Certifai scanner.**

**Task 1) Define the outcomes of the classification task**

In [10]:
# Define the possible prediction outcomes
task = CertifaiPredictionTask(CertifaiTaskOutcomes.classification(
    [
        CertifaiOutcomeValue(1, name='Loan granted', favorable=True),
        CertifaiOutcomeValue(2, name='Loan denied')
    ]),
    prediction_description='Determine whether a loan should be granted')

**Task 2) Create the Certifai scan object**

In [11]:
scan = CertifaiScanBuilder.create('test_user_case',
                                  prediction_task=task)

**Task 3) Create the Certifai dataset from the local dataset**

In [12]:
# Add the eval dataset
eval_dataset = CertifaiDataset('evaluation',
                               CertifaiDatasetSource.csv(all_data_file))

**Task 4) Create the Certifai model from the local model**


In [13]:
# Add our local model
first_model = CertifaiModel('logistic_regression',
                            local_predictor=logistic_model_proxy)
scan.add_model(first_model)

**Task 5) Setup an evaluation for fairness, robustness, and explainability on the above dataset using the model**

**We can have one or many of the following analysis types:**
- fairness
- robustness
- explainability
- explanation
- performance

**More information on these analyses can be found at our docs: https://cognitivescale.github.io/cortex-certifai/docs/information/factors/fairness**

In [14]:
scan.add_dataset(eval_dataset)
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('age'))
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('status'))
scan.add_evaluation_type('fairness')
scan.add_evaluation_type('explainability')
scan.add_evaluation_type('robustness')
scan.evaluation_dataset_id = 'evaluation'

**Task 6) Because the dataset contains a ground truth outcome column which the model does not expect to receive as input we need to state that in the dataset schema (since it cannot be inferred from the CSV) so that the scan can be rerun from the definition.**

In [15]:
scan.dataset_schema.outcome_feature_name = 'outcome'

**Task 7) Run the scan. 
    By default this will write the results into individual report files (one per model and evaluation
    type) in the 'reports' directory relative to the notebook.  This may be disabled by specifying
    `write_reports=False` as below**

In [16]:
#Run the Scan
result = scan.run(write_reports=False)

2023-01-09 12:01:43,917 root   WARNING  Insufficient examples of some fairness classes to guarantee convergence (smallest included class size is for status='male : divorced/separated' with 50 samples)


Starting scan with model_use_case_id: 'test_user_case' and scan_id: '5924c1cd0855'
[--------------------] 2023-01-09 12:01:43.907821 - 0 of 4 reports (0.0% complete) - Running fairness evaluation for model: logistic_regression


2023-01-09 12:02:16,452 root   WARNING  Examples of protected class ('status', 'male : divorced/separated') exhausted before convergence after 50 samples
2023-01-09 12:02:41,977 root   WARNING  Examples of protected class ('status', 'male : married/widowed') exhausted before convergence after 92 samples
2023-01-09 12:02:57,607 root   WARNING  Examples of protected class ('age', '<= 25 years') exhausted before convergence after 190 samples
2023-01-09 12:03:29,168 root   WARNING  Examples of protected class ('status', 'female : divorced/separated/married') exhausted before convergence after 310 samples
2023-01-09 12:03:33,063 root   WARNING  Examples of protected class ('age', '> 25 years') exhausted before convergence after 810 samples
2023-01-09 12:03:33,064 root   WARNING  Examples of protected class ('status', 'male : single') exhausted before convergence after 548 samples


[#####---------------] 2023-01-09 12:03:51.988622 - 1 of 4 reports (25.0% complete) - Running explainability evaluation for model: logistic_regression
[##########----------] 2023-01-09 12:05:06.843828 - 2 of 4 reports (50.0% complete) - Running robustness evaluation for model: logistic_regression
[###############-----] 2023-01-09 12:06:14.739678 - 3 of 4 reports (75.0% complete) - Running atx evaluation for model: logistic_regression
[####################] 2023-01-09 12:06:14.742022 - 4 of 4 reports (100.0% complete) - Completed all evaluations


**The result is a dictionary keyed on analysis, containing reports keyed on model id (in our case 'local')**

**We will be extracting the score information in the form of a DataFrame from this dictionary**

In [17]:
df = construct_scores_dataframe(scores('fairness', result), include_confidence=False)
display(df)

df = construct_scores_dataframe(scores('robustness', result), include_confidence=False)
display(df)

df = construct_scores_dataframe(scores('explainability', result), include_confidence=False)
display(df)

,context,type,fairness,Feature (age),type in Feature (age),Group details (<= 25 years),type in Feature (age),Group details (> 25 years),Feature (status),type in Feature (status),Group details (female : divorced/separated/married),type in Feature (status),Group details (male : divorced/separated),type in Feature (status),Group details (male : married/widowed),type in Feature (status),Group details (male : single)
logistic_regression (burden),logistic_regression,burden,68.744011,68.744011,burden,0.081351,burden,0.042567,72.706751,burden,0.074222,burden,0.074287,burden,0.032653,burden,0.036696


,context,robustness
logistic_regression,logistic_regression,86.935109


,context,explainability,Num features (1),Num features (10),Num features (2),Num features (3),Num features (4),Num features (5),Num features (6),Num features (7),Num features (8),Num features (9)
logistic_regression,logistic_regression,91.132812,47.65625,0.0,37.5,10.15625,3.90625,0.78125,0.0,0.0,0.0,0.0


# Step (3) Creating the exportable scan object
**Task 1) Next we'll make modify the scan definition to make it suitable for running against a version of the model deployed as a web service, and export this scan definition as a YAML file. We show how to  deploy the model as a web service and scan it using this scan definition file and the Certifai command line interface in Part 2 of this tutorial.**

**The two things that need to be changed are:**
- *predict_endpoint*: Since the model will be running in a web service, we need to provide the URL for its intended predict endpoint
- *dataset url*: Similarly, since the data will be read from persistent storage rather than an already populated DataFrame, we'll need to modify the data source accordingly. If the URL is a relative file path, it will be interpreted relative to where the scan definition is stored.

**Note that we could simply export the definition we have already, and add these fields to the resulting YAML (or have the deploying engineer do so), but it's a bit friendlier if we create placeholders that can be replaced later**

In [18]:
scan.models[0].predict_endpoint = 'http://mymodel/predict'
scan.datasets[0].source = CertifaiDatasetSource.csv('somefile.csv')

**The scan object contains the scan definition, which consists of all of the metadata needed to rerun the scan**

**Task 2) Viewing the scan definition**

In [19]:
print(scan.extract_yaml())

evaluation:
  evaluation_dataset_id: evaluation
  evaluation_types:
  - fairness
  - explainability
  - robustness
  fairness_grouping_features:
  - name: age
  - name: status
  prediction_values:
  - name: Loan granted
    value: 1
    favorable: true
  - name: Loan denied
    value: 2
    favorable: false
  prediction_favorability: explicit
  name: test_user_case
  prediction_description: Determine whether a loan should be granted
model_use_case:
  name: test_user_case
  model_use_case_id: test_user_case
  task_type: binary-classification
dataset_schema:
  outcome_column: outcome
datasets:
- dataset_id: evaluation
  url: somefile.csv
  has_header: true
  file_type: csv
  delimiter: ','
  quote_character: '"'
models:
- model_id: logistic_regression
  name: logistic_regression
  predict_endpoint: http://mymodel/predict
  prediction_value_order:
  - 1
  - 2



**Task 3) Save the Scan Definition locally.**

**Save the scan definition to a file. The file path is relative to the notebook**

In [20]:
scan_file="./scan_definition.yaml"
with open(scan_file, "w") as f:
    scan.save(f)
    print(f"Saved template to: {scan_file}")

Saved template to: ./scan_definition.yaml


**We will see how to use this definition to kickstart scans in the CLI in part 2 of this tutorial.**